# *Import Packages*

In [9]:
!pip install contractions
!pip install nltk
import contractions                
import nltk      
nltk.download('stopwords')           
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')           
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer        
from nltk.tokenize import word_tokenize, sent_tokenize 
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import re
from nltk.stem import PorterStemmer
!pip install torchmetrics
from keras.preprocessing.text import Tokenizer
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# *Read Dataset (Product Catalog of Flipkart)*

In [17]:
df = pd.read_csv('/content/ecommerce_dataset.csv')
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


# *Data Cleaning & Feature Engineering*

In [18]:
data_category = df.product_category_tree.str.split(" >> ", expand=True)
for x in range(8):
    data_category[x] = data_category[x].str.replace("\[\"","")
    data_category[x] = data_category[x].str.replace("\"\]","")

df["category"] = data_category[0];
categories = np.concatenate([data_category[0], data_category[1], data_category[2],data_category[3],data_category[4],data_category[5],data_category[6],data_category[7]])
categories = list(dict.fromkeys(categories))

categories = [category for category in categories if x is not None]

data_category = pd.concat([data_category,pd.DataFrame(columns = list(categories))])

data_category.fillna(0, inplace = True)

#We have maximum 7 categories in Category tree. 
#We will set 1 to consider our category as a class
for i in range(7):
  counter = 0
  for category in data_category[i]:
    if category != 0:
      data_category.loc[counter,category] = 1
    counter = counter + 1

data_category_classes = pd.concat([df['product_name'],df['description'],data_category.loc[:,'Clothing':]],axis=1) 
data_category_classes.shape

<ipython-input-18-2a8064975868>:3: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-18-2a8064975868>:4: FutureWarning:

The default value of regex will change from True to False in a future version.

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:2955: RuntimeWarning:

'<' not supported between instances of 'NoneType' and 'int', sort order is undefined for incomparable objects



(20000, 9048)

Remove categories which are not that common

In [19]:
#We need to set threshold to remove unnecessary categories and subcategories
count_plot = pd.DataFrame()
count_plot['category'] = data_category_classes.columns[2:] 
count_plot['count'] = data_category_classes.iloc[:,2:].sum().values
count_plot.sort_values(['count'], inplace=True, ascending=False)

#Set thresold 
threshold = 5

major_categories = pd.DataFrame()
major_categories = count_plot[count_plot['count'] > threshold]
categories = major_categories['category'].values

unnecessary_categories = []

for i in data_category_classes.columns[2:]:
    if i not in categories:
        unnecessary_categories.append(i)

data_category_classes.drop(unnecessary_categories, axis=1, inplace=True)
print(data_category_classes.head())

                            product_name  \
0    Alisha Solid Women's Cycling Shorts   
1    FabHomeDecor Fabric Double Sofa Bed   
2                             AW Bellies   
3    Alisha Solid Women's Cycling Shorts   
4  Sicons All Purpose Arnica Dog Shampoo   

                                         description  Clothing  Furniture  \
0  Key Features of Alisha Solid Women's Cycling S...         1          0   
1  FabHomeDecor Fabric Double Sofa Bed (Finish Co...         0          1   
2  Key Features of AW Bellies Sandals Wedges Heel...         0          0   
3  Key Features of Alisha Solid Women's Cycling S...         1          0   
4  Specifications of Sicons All Purpose Arnica Do...         0          0   

   Footwear  Pet Supplies  Pens & Stationery  Sports & Fitness  \
0         0             0                  0                 0   
1         0             0                  0                 0   
2         1             0                  0                 0   
3       

# *Data Visualization*

# Historgram - To check which brand is having more number of products

In [6]:
#Data Cleaning operations
data_footwear = df.loc[df.loc[:,'category'] == 'Footwear']
data_footwear.loc[:,'product_rating'] = data_footwear.loc[:,'product_rating'].replace('No rating available',0)
data_footwear.loc[:,'product_rating'] = data_footwear.loc[:,'product_rating'].astype(float)
data_footwear.loc[:,'discounted_price'] = data_footwear.loc[:,'discounted_price'].fillna(0)
data_footwear.loc[:,'discounted_price'] = data_footwear.loc[:,'discounted_price'].astype(float)

fig = px.histogram(data_footwear, x='brand', color="category", barmode='group')

fig.show()



/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


# ScatterPlot - To check which brand is having more rating

In [ ]:
fig = px.scatter(data_footwear, x='brand', y="product_rating",color="product_rating",
                    size = 'product_rating',hover_data=['product_name'])

fig.show()

# ScatterPlot - To check which brand is having more discount price

In [ ]:
fig = px.scatter(data_footwear, x='brand', y="discounted_price",color="discounted_price",
                    size = 'discounted_price',hover_data=['product_name'])

fig.show()

# *Text Cleaning for Product Description*

In [21]:
#Remove punctuations 
def removePunctuations(input): 
    input = str(input)
    processed_input = re.sub(r'[?|!|\'|"|#|.|,|)|(|\|/]',r'',input)
    processed_input = processed_input.strip().replace("\n"," ")
    return processed_input

#Expand Contractions - We'll -> We will
def expandContractions(input):
    input = str(input)
    input = contractions.fix(input)
    return input

#Keep alphabetic characters
def alphacharacters(input):
    input = str(input)
    processed_word = ""
    for word in input.split():
        word = re.sub('[^a-z A-Z]+', '', word)
        processed_word += word
        processed_word += " "
    processed_word = processed_word.strip()
    return processed_word

#Remove stopwords
def removeStopWords(input):
    input = str(input)
    processed_input = []
    stop_words = set(stopwords.words('english'))
    for word in input.split():
      if word not in stop_words:
        processed_input.append(word)
    return processed_input

#Remove similar form words
snowball_stemmer = SnowballStemmer("english")
def stemming(input):
    input = str(input)
    processed_input = ""
    for word in input.split():
        stem = snowball_stemmer.stem(word)
        processed_input += stem
        processed_input += " "
    processed_input = processed_input.strip()
    return processed_input

#removePunctuations(text)
#expandContractions(text1)
#alphacharacters(text1)
#removeStopWords(text1)
#stemming(text)

# Apply Text Cleaning for Product Title and Product Description


In [22]:
final_df = data_category_classes
print (final_df.head())
#Apply Text cleaning operations on Product Name
final_df['product_name'] = final_df['product_name'].str.lower()
final_df['product_name'] = final_df['product_name'].apply(expandContractions)
final_df['product_name'] = final_df['product_name'].apply(removePunctuations)
final_df['product_name'] = final_df['product_name'].apply(alphacharacters)
final_df['product_name'] = final_df['product_name'].apply(removeStopWords)
final_df['product_name'] = final_df['product_name'].apply(stemming)

#Apply Text cleaning on Product Description
final_df['description'] = final_df['description'].str.lower()
final_df['description'] = final_df['description'].apply(expandContractions)
final_df['description'] = final_df['description'].apply(removePunctuations)
final_df['description'] = final_df['description'].apply(alphacharacters)
final_df['description'] = final_df['description'].apply(removeStopWords)
final_df['description'] = final_df['description'].apply(stemming)

final_df['data'] = final_df['product_name'] + final_df['description']

final_df.head()

                            product_name  \
0    Alisha Solid Women's Cycling Shorts   
1    FabHomeDecor Fabric Double Sofa Bed   
2                             AW Bellies   
3    Alisha Solid Women's Cycling Shorts   
4  Sicons All Purpose Arnica Dog Shampoo   

                                         description  Clothing  Furniture  \
0  Key Features of Alisha Solid Women's Cycling S...         1          0   
1  FabHomeDecor Fabric Double Sofa Bed (Finish Co...         0          1   
2  Key Features of AW Bellies Sandals Wedges Heel...         0          0   
3  Key Features of Alisha Solid Women's Cycling S...         1          0   
4  Specifications of Sicons All Purpose Arnica Do...         0          0   

   Footwear  Pet Supplies  Pens & Stationery  Sports & Fitness  \
0         0             0                  0                 0   
1         0             0                  0                 0   
2         1             0                  0                 0   
3       

,product_name,description,Clothing,Furniture,Footwear,Pet Supplies,Pens & Stationery,Sports & Fitness,Beauty and Personal Care,"Bags, Wallets & Belts",...,Babeezworld Baby Boy's Romper,Satovira Party Sleeveless Solid Women's Top,Jazzup Boy's Kurta and Pyjama Set,next steps Baby Girl's Printed Top & Pyjama Set,Mothercare Printed Round Neck T-Shirt,"fashion rudraksh Embroidered Women's Lehenga, Ch...",Cotton Fab Printed Women's Maternity Wear Kurti ...,Bodycare Checkered Boy's Boxer (Pack of 2),carrol Women's Shrug,data
0,alisha solid women cycl short,key featur alisha solid women cycl short cotto...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alisha solid women cycl shortkey featur alisha...
1,fabhomedecor fabric doubl sofa bed,fabhomedecor fabric doubl sofa bed finish colo...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fabhomedecor fabric doubl sofa bedfabhomedecor...
2,aw belli,key featur aw belli sandal wedg heel casual aw...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,aw bellikey featur aw belli sandal wedg heel c...
3,alisha solid women cycl short,key featur alisha solid women cycl short cotto...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alisha solid women cycl shortkey featur alisha...
4,sicon purpos arnica dog shampoo,specif sicon purpos arnica dog shampoo ml gene...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,sicon purpos arnica dog shampoospecif sicon pu...




---



---



# *Model Implementation*

# Deep Learning - PyTorch

In [ ]:
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(final_df['data'])
datasequences = tokenizer.texts_to_sequences(final_df['data'])
x = pad_sequences(datasequences, maxlen=500)
y = final_df[final_df.columns[2:-1]].to_numpy()
print (x)
print (y)

[[ 0  0  0 ... 19 20 25]
 [ 0  0  0 ... 19 20 25]
 [ 0  0  0 ... 19 20 25]
 ...
 [ 0  0  0 ... 19 20 25]
 [ 0  0  0 ... 19 20 25]
 [ 0  0  0 ... 19 20 25]]
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,  
                                                    test_size=0.1, 
                                                    random_state=0)
num_class = len(y_train[0])
input_dimension = 500
print(num_class) #Number of categories(include sub-categories)
print(len(X_train))

925
18000


In [ ]:
# PyTorch models inherit from torch.nn.Module
class Classifier1(nn.Module):
    def __init__(self):
        super(Classifier1, self).__init__()
        self.fc1 = nn.Linear(500, 600, dtype=torch.float64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(600, 800, dtype=torch.float64)
        self.fc3 = nn.Linear(800, 925, dtype=torch.float64)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x


FN_model = Classifier1().cuda()

In [ ]:
test = torch.rand((256,500)).to(torch.float64).cuda()
print(FN_model(test).shape)

torch.Size([256, 925])


In [ ]:
loss_fn = nn.BCELoss()
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 6:
        lr = 1e-4
    elif epoch < 9:
        lr = 1e-5
    else:
        lr = 1e-6

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).to(torch.float64)
        self.y = torch.tensor(y).to(torch.float64)
    
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, index):
        data = self.x[index]
        label = self.y[index]
        return data, label
dataset = CustomDataset(X_train,y_train)
Dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=3,
                          pin_memory=False, drop_last=False)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
def read_data(data):
  return (data[0].cuda(),data[1].cuda())

def train(model, Dataloader, loss_fn, epochs):

    optimizer = get_optimizer(model)

    for e in range(epochs):
        model.train()
        tbar = tqdm(Dataloader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []

        for idx, data in enumerate(tbar):
            data = read_data(data)
            inputs = data[0]
            target = data[1]

            optimizer.zero_grad()
            
            logits = model(inputs)
            loss = loss_fn(logits, target)
            
            loss.backward()
            optimizer.step()
            
            loss_list.append(loss.detach().item())
            
            avg_loss = np.round(100*np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
    return model

In [ ]:
FN_model = train(FN_model, Dataloader, loss_fn, 10)

Epoch 10 Loss: 23.0102 lr: 1e-06: 100%|██████████| 71/71 [00:01<00:00, 40.77it/s]


In [ ]:
#Evaluation
dataset = CustomDataset(X_test,y_test)
Dataloader = DataLoader(dataset, batch_size=len(X_test), shuffle=False, num_workers=3,
                          pin_memory=False, drop_last=False)
from torchmetrics.classification import BinaryAUROC
def validate(model, val_loader):
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    maps = []
    metric = BinaryAUROC(thresholds=None)
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs = data[0]
            target = data[1]
            model = model.cpu()
            logits = model(inputs)

            for i in range(logits.shape[0]):
                  maps.append(metric(logits[i], target[i]))
        
    return np.mean(maps)
accuracy1 = validate(FN_model,Dataloader)
print('*'*30)
print('The model only uses fully connected layer: ',accuracy1)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


100%|██████████| 1/1 [00:01<00:00,  1.86s/it]
******************************
The model only uses fully connected layer:  0.7950274538397789


In [ ]:
del FN_model



---



---



# ConvNet Model - Implementation

In [ ]:
class Classifier2(nn.Module):
    def __init__(self,max_words,dim):
        super(Classifier2, self).__init__()
        self.emb = nn.Embedding(max_words, embedding_dim=dim)
        self.fc1 = nn.Linear(200, 925, dtype=torch.float64)
        self.sigmoid = nn.Sigmoid()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.top = nn.Sequential(nn.Conv1d(500, 200, kernel_size=1, dtype=torch.float64), nn.LeakyReLU(),nn.BatchNorm1d(200, dtype=torch.float64),
                                 nn.MaxPool1d(kernel_size=300))
        

    def forward(self, x):
        x = self.emb(x.to(torch.long)).to(torch.float64)
        x = self.flatten(self.top(x))
        x = self.relu(self.fc1(x))
        x = self.sigmoid(x)
        return x

Conv_model = Classifier2(5000,300).cuda()

In [ ]:
loss_fn = nn.BCELoss()
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 2:
        lr = 1e-4
    elif epoch < 3:
        lr = 5e-5
    elif epoch < 4:
        lr = 1e-6
    else:
        lr = 1e-7

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

In [ ]:
dataset = CustomDataset(X_train,y_train)
Dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=3,
                          pin_memory=False, drop_last=False)
Conv_model = train(Conv_model, Dataloader, loss_fn, 5)

Epoch 5 Loss: 69.3638 lr: 1e-07: 100%|██████████| 71/71 [00:47<00:00,  1.49it/s]


In [ ]:
dataset = CustomDataset(X_test,y_test)
Dataloader = DataLoader(dataset, batch_size=len(X_test), shuffle=False, num_workers=3,
                          pin_memory=False, drop_last=False)
accuracy2 = validate(Conv_model,Dataloader)
print('*'*30)
print('The model uses Convolutional layer and fully connected layer: ',accuracy2)

100%|██████████| 1/1 [00:11<00:00, 11.78s/it]
******************************
The model uses Convolutional layer and fully connected layer:  0.49548680886626245


In [ ]:
del Conv_model



---



---



# Logistic Regression Model(OnevsRest Technique - Implementation)

In [23]:
X = final_df['data']
y = final_df[final_df.columns[2:-1]]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20)
print (np.shape(X_train))
print (np.shape(y_train))

vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2') 
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

(16000,)
(16000, 925)


In [24]:
LogisticRegressor = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))]) 

accuracy = 0
auc = 0
counter = 1
#Iterate for each category and apply Logistic Regression model on it
for category in final_df.columns[2:-1]:    
    LogisticRegressor.fit(X_train, y_train[category]) 
    prediction = LogisticRegressor.predict(X_test)  
    score = accuracy_score(y_test[category], prediction)
    accuracy = accuracy + score
    
    print('Product Category => {} '.format(category)," Accuracy => {}".format(score))
    file = "data1/"+ str(counter)+"_logisticmodel.sav"
    joblib.dump(LogisticRegressor, file)        
    counter += 1

print('Average Accuracy => {}'.format(accuracy/len(final_df.columns[2:-1])))

Product Category => Clothing   Accuracy => 0.99
Product Category => Furniture   Accuracy => 0.99625
Product Category => Footwear   Accuracy => 0.991
Product Category => Pet Supplies   Accuracy => 0.99825
Product Category => Pens & Stationery   Accuracy => 0.99175
Product Category => Sports & Fitness   Accuracy => 0.991
Product Category => Beauty and Personal Care   Accuracy => 0.9875
Product Category => Bags, Wallets & Belts   Accuracy => 0.994
Product Category => Home Decor & Festive Needs   Accuracy => 0.98975
Product Category => Automotive   Accuracy => 0.99425
Product Category => Tools & Hardware   Accuracy => 0.99675
Product Category => Vishudh Printed Women's Straight Kurta   Accuracy => 1.0
Product Category => Home Furnishing   Accuracy => 0.99025
Product Category => Baby Care   Accuracy => 0.99275
Product Category => Mobiles & Accessories   Accuracy => 0.99125
Product Category => Watches   Accuracy => 0.999
Product Category => Toys & School Supplies   Accuracy => 0.99
Product C

In [33]:
def recommendCategories(user_input):

  #Apply NLP Operations over the user statement
  statement = user_input.lower()
  statement = expandContractions(statement)
  statement = removePunctuations(statement)
  statement = alphacharacters(statement)
  statement = removeStopWords(statement)
  statement = stemming(statement)

  X_api = vectorizer.transform([statement])

  accuracy = 0
  counter = 1

  #Calling the model to predict category
  for category in final_df.columns[2:-1]:
    filename = "data1/" + str(counter)+"_logisticmodel.sav"
    counter += 1
    LR_pipeline = joblib.load(filename)

    prediction = LR_pipeline.predict(X_api)
    if prediction==1:
      print('CATEGORY {}'.format(category))



---



---



# Recommend Products - Final Output Prediction

In [34]:
statement = "Alisha Solid Women's Cycling Shorts"
description = "Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"
recommendCategories(statement)

CATEGORY Clothing
CATEGORY Women's Clothing


In [35]:
statement = "AW Bellies"
recommendCategories(statement)

In [36]:
statement = "For Women Occasion Casual Shoe Patent Leather Heel"
recommendCategories(statement)

CATEGORY Footwear


In [37]:
name = "Women Shorts"
statement = "Women Cycling Shorts"
recommendCategories(statement)

CATEGORY Clothing
CATEGORY Women's Clothing


In [38]:
name = "White Tshirt"
statement = "White Watches"
recommendCategories(statement)

In [39]:
name = "RR-030 Essentials Analog Watch You will be bombarded with complimenting glances as you walk out wearing this black coloured analog watch"
statement = "White kurti"
recommendCategories(statement)

CATEGORY Clothing
CATEGORY Ethnic Wear
CATEGORY Kurtas & Kurtis
CATEGORY Kurtis


In [40]:
statement =  "White Tshirt round neck"
recommendCategories(statement)

CATEGORY Clothing


In [41]:
statement = "Gold Diamond Ring"
recommendCategories(statement)

CATEGORY Jewellery
CATEGORY Rings


In [ ]:
!zip -r /content/file.zip /content/Folder_To_Zip

from google.colab import files
files.download("/content/file.zip")